Practice problems
=================



This directory is designed to be "org-published". The idea is to have org files that setup practice problems with links to solutions.



In [1]:
(defun ox-ipynb-publish-practice (plist filename pub-dir)
  "Publish an org-file to a Jupyter notebook."
  (let ((buf-open (find-buffer-visiting filename))
	(buf (find-file-noselect filename))
	(ipynb (file-name-nondirectory (concat (file-name-sans-extension filename) ".ipynb"))))

    (with-current-buffer buf
      (let* ((ox-ipynb-preprocess-hook
	      (append ox-ipynb-preprocess-hook
		      `((lambda ()
			  (goto-char (point-min))
			  (insert (format "[[https://colab.research.google.com/github/jkitchin/f21-06623/blob/master/practice/notebooks/%s][Open in colab]]\n\n"
					  ,(file-name-nondirectory (concat (file-name-sans-extension filename) ".ipynb"))))
			  (goto-char (point-max))

			  (unless (member filename '("readme.org", "sitemap.org"))
			    (insert (format "\n\n* Solutions\n\n [[https://colab.research.google.com/github/jkitchin/f21-06623/blob/master/practice/solutions/%s][Open in colab]]\n\n"
					    ,(file-name-nondirectory (concat (file-name-sans-extension filename) ".ipynb"))))))

			ox-ipynb-remove-hidden
			ox-ipynb-remove-solution
			ox-ipynb-remove-remove)))
	     (output (ox-ipynb-export-to-ipynb-file)))
	(org-publish-attachment plist (expand-file-name output) pub-dir)
	(unless buf-open (kill-buffer buf))

	(delete-file ipynb)
	output))))


(defun ox-ipynb-publish-solution (plist filename pub-dir)
  "Publish an org-file to a Jupyter notebook."
  (let ((buf-open (find-buffer-visiting filename))
	(buf (find-file-noselect filename))
	(ipynb (file-name-nondirectory (concat (file-name-sans-extension filename) ".ipynb"))))
    (with-current-buffer buf
      (let* ((output (ox-ipynb-export-to-ipynb-file)))
	(org-publish-attachment plist (expand-file-name output) pub-dir)
	(unless buf-open (kill-buffer buf))

	(delete-file ipynb)
	output))))


(defun f21-06623-index ()
  (concat "* Index\n\n"
	  (let ((index-entries '()))
	    (cl-loop for org-file in (f-files "." (lambda (f) (f-ext? f "org"))) do
		     (with-current-buffer (find-file-noselect org-file)
		       (cl-loop for  entry in (cdar (org-collect-keywords '("INDEX"))) do
				(push (cons entry (concat (file-name-sans-extension (file-name-nondirectory org-file)) ".ipynb"))
				      index-entries))))
	    ;; sorted and grouped
	    (cl-loop for group in
		     (seq-group-by (lambda (x) (substring (car x) 0 1)) (sort index-entries (lambda (a b) (string< (car a) (car b)))))
		     concat
		     (format "** %s\n\n%s\n" (car group)
			     (mapconcat (lambda (s) (format " [[./%s][%s]]\n" (cdr s) (car s))) (cdr group) "\n"))))))

(setq org-publish-project-alist
      '(("notebooks"
         :base-directory "/Volumes/GoogleDrive/My Drive/classes/f21-06623/f21-06623/f21-06623/practice/"
         :base-extension "org"
         :publishing-directory "/Volumes/GoogleDrive/My Drive/classes/f21-06623/f21-06623/f21-06623/practice/notebooks"
	 :auto-sitemap t
	 :sitemap-format-entry (lambda (fname style project)
				 (format "[[./%s][%s]]%s"
					 (concat (file-name-sans-extension fname) ".ipynb")
					 (concat (file-name-sans-extension fname) ".ipynb")
					 (with-current-buffer (find-file-noselect fname)
					   (if (org-get-buffer-tags)
					       (format " (%s)"
						       (org-no-properties (mapconcat 'org-element-interpret-data (org-get-buffer-tags) ", ")))
					     ""))))
	 :sitemap-function (lambda (title files)
			     (format "#+TITLE: %s

%s

%s"
				     title
				     (string-join (cl-loop for entry in (cdr files) collect (concat "- " (car entry))) "\n")
				     (f21-06623-index)))

	 :publishing-function  ox-ipynb-publish-practice)

	("solutions"
	 :base-directory "/Volumes/GoogleDrive/My Drive/classes/f21-06623/f21-06623/f21-06623/practice/"
         :base-extension "org"
         :publishing-directory "/Volumes/GoogleDrive/My Drive/classes/f21-06623/f21-06623/f21-06623/practice/solutions"
	 :publishing-function ox-ipynb-publish-solution
	 :sitemap-format-entry (lambda (fname style project)
				 (format "[[./%s][%s]]%s"
					 (concat (file-name-sans-extension fname) ".ipynb")
					 (concat (file-name-sans-extension fname) ".ipynb")
					 (with-current-buffer (find-file-noselect fname)
					   (if (org-get-buffer-tags)
					       (format " (%s)"
						       (org-no-properties (mapconcat 'org-element-interpret-data (org-get-buffer-tags) ", ")))
					     ""))))
	 :sitemap-function (lambda (title files)
			     (format "#+TITLE:%s

%s

%s"
				     title
				     (string-join (cl-loop for entry in (cdr files) collect (concat "- " (car entry))) "\n")
				     (f21-06623-index)))


	 )

	("practice" :components ("notebooks" "solutions"))))


(org-publish "practice")

(shell-command "git config advice.addIgnoredFile false")
(shell-command "git add *")
(shell-command "git commit -am \"published updates\"")
(shell-command "git push")
(message "done")